# Georeference images based on a pair [id ; extent]

In [1]:
import rasterio
from rasterio.transform import from_origin
from PIL import Image
import imageio
import numpy as np
import os

def calculate_pixel_size(extent, width, height):
    pixel_size_x = (extent[2] - extent[0]) / width
    pixel_size_y = (extent[3] - extent[1]) / height
    return pixel_size_x, pixel_size_y

def get_number_of_bands(image_path):
    try:
        reader = imageio.get_reader(image_path)
        num_bands = reader.get_data(0).shape[-1]
        reader.close()
        return num_bands

    except Exception as e:
        return None

def create_georeferenced_raster(image_path, out_dir, extent):
    # Make output directory
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    # Read img
    img = imageio.imread(image_path)

    # Georeference it
    height, width, _ = img.shape
    pixel_size_x, pixel_size_y = calculate_pixel_size(extent, width, height)

    transform = from_origin(extent[0], extent[3], pixel_size_x, pixel_size_y)

    # Save it
    filename = os.path.basename(image_path).split(".")[0]

    with rasterio.open(f'{out_dir}/{filename}.tiff', 'w', driver='GTiff', height=height, width=width, count=get_number_of_bands(image_path), dtype=str(img.dtype), crs='EPSG:4326', transform=transform) as dst:
        dst.write(img.transpose(2, 0, 1))


In [2]:
import pandas as pd
from ast import literal_eval

csv_file_path = 'pairs.csv'

df = pd.read_csv(csv_file_path, sep=";")
df['extent'] = df['extent'].apply(literal_eval)

print(df)

for x in range(len(df)):
    img_id = df.iloc[x]["id"]
    extent = df.iloc[x]["extent"]

    in_dir = "imgs"
    out_dir = 'georefs'

    create_georeferenced_raster(f'{in_dir}/{img_id}.png', out_dir, extent)


   id                                    extent
0   0  [0.442059, 0.368963, 0.658653, 0.489495]
1   1   [0.715263, 0.310654, 1.005402, 0.51112]
2   2  [0.692274, 0.183193, 0.856452, 0.281886]
3   3  [0.865723, 0.054862, 0.983933, 0.198455]
4   4  [0.390131, 0.077752, 0.656987, 0.344609]
5   5   [0.200876, 0.10643, 0.337549, 0.220162]
6   6   [0.122379, 0.226932, 0.35506, 0.555998]
7   7  [0.039566, 0.569517, 0.363965, 0.737914]
8   8   [0.38198, 0.528808, 0.700066, 0.753732]
9   9    [0.755275, 0.568778, 0.9006, 0.819824]


C:\Users\nerii\AppData\Local\Temp\ipykernel_9996\535626829.py:29: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imageio.imread(image_path)
c:\Users\nerii\Desktop\Work\DVS\Multiplicity\website\2023-24\processing\env_multiplicity\lib\site-packages\PIL\Image.py:3186: DecompressionBombWarning: Image size (128320830 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
